# Predicting University Admissions
In this jupyter notebook I will predict university admissions based
on a dataset found here: https://www.kaggle.com/tanmoyie/us-graduate-schools-admission-parameters.

All the methods implemented from scratch use mini-batch stochastic gradient descent while all the scikit learn
methods use regular stochastic gradient descent.

From scratch:
1. Linear Regression
    * Fixed step size of 0.01
    * Heuristic step size
    * Step size determined with Adagrad
2. Polynomial Regression
    * Fixed step size of 0.01
    * Heuristic step size
    * Step size determined with Adagrad

With scikit-learn:
1. Linear Regression
    * Fixed step size of 0.01
    * Inversely scaled step size
2. Ridge Regression
    * Fixed step size of 0.01
    * Inversely scaled step size
3. Lasso Regression
    * Fixed step size of 0.01
    * Inversely scaled step size
3. Polynomial Regression
    * Fixed step size of 0.01
    * Inversely scaled step size
4. Polynomial Ridge Regression
5. Polynomial Lasso Regression

## Exploring the Data

In [ ]:
import pandas as pd
data = pd.read_csv('AdmissionsData.csv')
data.head()

In [ ]:
data.describe()

In the next part we are interested in the last column because it tells us if each individual feature is
positively correlated to the chance of admission. In other words we can check if the feature can help us predict the
chance of admission. We can see that GRE Score, CGPA, and TOEFL Score all strongly correlate with the chance of admission.

In [ ]:
data.corr()

We can further explore the relationship between variables using scatter plots to visualize correlations and outliers.

In [ ]:
data.plot(x=data.columns[-1], y='GRE Score', kind='scatter')
data.plot(x='GRE Score', y='TOEFL Score', kind='scatter')
data.plot(x='GRE Score', y='CGPA', kind='scatter')

## Reading and cleaning the data
There is no data cleaning to do since this dataset came from Kaggle and is already cleaned.
##### Load the data

In [ ]:
import numpy as np

np.random.seed(1)

filename = 'AdmissionsData.csv'

# [1:, 1:] removes the first row (column names) and the first column (index)
dataset = np.genfromtxt(filename, delimiter=',')[1:, 1:]
print(dataset[:5])

To make predictions with only the GRE Score, CGPA, and TOEFL Score features run the code block below.

In [ ]:
# for i in range(3):
#     dataset = np.delete(dataset, 2, 1)
# dataset = np.delete(dataset, 3, 1)


##### Split the data into training and test sets

In [ ]:
import random
import numpy as np

numFeatures = dataset.shape[1] - 1
randomIndices = np.random.choice(dataset.shape[0], len(dataset), replace=False)
Xtrain = dataset[randomIndices[:350], :numFeatures]
Ytrain = dataset[randomIndices[:350], numFeatures]

Xtest = dataset[randomIndices[350:], :numFeatures]
Ytest = dataset[randomIndices[350:], numFeatures]

##### Normalize the features

In [ ]:
for i in range(Xtrain.shape[1]):
    Xtrain[:, i] = np.divide(Xtrain[:, i], np.max(np.abs(Xtrain[:, i])))
    Xtest[:, i] = np.divide(Xtest[:, i], np.max(np.abs(Xtest[:, i])))

##### Augment data with column of 1's (acts as intercept weight)

In [ ]:
Xtrain = np.hstack((Xtrain, np.ones((Xtrain.shape[0], 1))))
Xtest = np.hstack((Xtest, np.ones((Xtest.shape[0], 1))))

***
# Linear Regression
The following methods all use mini-batch stochastic gradient descent but have distinct
methods of determining step size.

##### Initialize values

In [ ]:
batchSize = 50
epochs = 200

## Train

In [ ]:
def trainLinReg(Xtrain, Ytrain, stepSizeChoice):
    numSamples = Xtrain.shape[0]
    numFeatures = Xtrain.shape[1]
    weights = np.zeros(numFeatures)

    if stepSizeChoice == "heuristic":
        gBar = 1
    elif stepSizeChoice == "adagrad":
        gBar = np.zeros(numFeatures)

    # loop through epochs
    for i in range(epochs):
        shuffle = list(zip(Xtrain, Ytrain))
        random.shuffle(shuffle)
        X, Y = zip(*shuffle)
        Xtrain = np.array(list(X))
        Ytrain = np.array(list(Y))

        index = 0
        for j in range(numSamples//batchSize):
            gradient = np.zeros(numFeatures)

            # calculate the gradient for a given batch
            for x in range(index, index + batchSize):
                # (xw - y)x sgd update rule
                gradient += np.dot(np.dot(Xtrain[x], weights) - Ytrain[x], Xtrain[x])
            gradient = (1 / batchSize) * gradient

            # choose step size
            if stepSizeChoice == "heuristic":
                gBar = gBar + ((1/8) * np.sum(np.abs(gradient)))
                stepSize = (1 + gBar) ** (-1)
            elif stepSizeChoice == "adagrad":
                gBar = gBar + np.square(gradient)
                stepSize = np.divide(np.ones(gBar.shape[0]), np.sqrt(gBar))
            else:
                stepSize = 0.01

            # update weights
            weights = weights - np.multiply(stepSize, gradient)

            index += batchSize

    return weights

***
## Predict and Calculate Error

In [ ]:
meanTrain = np.mean(Ytrain)
meanPredict = np.ones((Xtest.shape[0],)) * meanTrain
meanErr = np.square(np.subtract(meanPredict, Ytest)).mean()
print('Baseline Mean Squared Error:', meanErr)

linRegErrors = []
stepType = ['fixed', 'heuristic', 'adagrad']
for step in stepType:
    weights = trainLinReg(Xtrain, Ytrain, step)
    predictions = np.dot(Xtest, weights)
    linRegErrors.append(np.subtract(predictions, Ytest))
    error = np.square(np.subtract(predictions, Ytest)).mean()
    print(f"Mean Squared Error with {step} step size: ", error)

***
# Polynomial Regression
## Train

In [ ]:
# transforms Xtrain data to fit a polynomial with degree 2
def transform(Xdata):
    x = list(Xdata)
    x.reverse()
    # loop starting from index 1 since the first term (1) is a bias term that isn't an actual feature
    new_x = [i for i in x]
    for i in range(1, len(x)):
        new_x.append(x[i]**2)
        for j in range(i+1, len(x)):
            new_x.append(x[i] * x[j])
    return np.array(new_x)


polyXtrain = []
for i in range(len(Xtrain)):
    polyXtrain.append(transform(Xtrain[i]))
polyXtrain = np.array(polyXtrain)

polyXtest = []
for i in range(len(Xtest)):
    polyXtest.append(transform(Xtest[i]))
polyXtest = np.array(polyXtest)


## Predict and Calculate Error

In [ ]:
meanTrain = np.mean(Ytrain)
meanPredict = np.ones((polyXtest.shape[0],)) * meanTrain
meanErr = np.square(np.subtract(meanPredict, Ytest)).mean()
print('Baseline Mean Squared Error:', meanErr)


stepType = ['fixed', 'heuristic', 'adagrad']
errors = []
for step in stepType:
    weights = trainLinReg(polyXtrain, Ytrain, step)
    predictions = np.dot(polyXtest, weights)
    errors.append(np.subtract(predictions, Ytest))
    MSE = np.square(np.subtract(predictions, Ytest)).mean()
    print(f"Mean Squared Error with {step} step size: ", MSE)

## Error Analysis

In [ ]:
import matplotlib.pyplot as plt

for err in errors:
    plt.hist(err, bins=70)
    plt.show()

We can compare models and pick the best one with a certain level of confidence using the paired t-test.
Less run an example paired t-test to compare polynomial models trained with fixed vs Adagrad step size.

Our null hypothesis is that the performance of the fixed step size model is the same as the performance of the Adagrad
step size model (no significant difference in error). The alternative hypothesis is that the Adagrad model performed
better (significant difference in error).

We are assuming the data is i.i.d. and as we can see from the error charts above the errors are approximately normally
distributed. Furthermore, the mean and variance of the fixed step size error and Adagrad step size error are relatively
the same. Thus, we can calculate the t-statistic and subsequent p-value to determine which hypothesis is more probable.

In [ ]:
import math
from scipy.stats import t

m = len(errors[0])  # dimension of the test set
difference = np.subtract(linRegErrors[0], errors[2])
d = (1 / m) * np.sum(difference)
y = np.full(difference.shape, d)
temp = (1 / (m - 1)) * np.sum(np.square(np.subtract(difference, y)))

s = math.sqrt(temp)
k = d / (s / math.sqrt(m))

pVal = 1 - t.cdf(k, m-1)

print(pVal)

Since the p-value is > 0.05 we cannot say with 95% confidence that the difference in performance between Adagrad and
fixed step size models is significant. This was simply an example and much more rigorous analysis is necessary.

***
# Scikit-Learn
## Reading and cleaning the data
##### Load data

In [ ]:
from numpy import genfromtxt
filename = 'AdmissionsData.csv'
dataset = genfromtxt(filename, delimiter=',')[1:, 1:]
X = dataset[:, :7]
Y = np.concatenate(dataset[:, 7:], axis=0)

##### Split data

In [ ]:
from sklearn.model_selection import train_test_split

Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, train_size=0.7, shuffle=True)
print(Xtrain[:5])
print(Ytrain[:5])

##### Normalize

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
Xtrain = scaler.fit_transform(Xtrain)
Xtest = scaler.fit_transform(Xtest)
print(Xtrain[:5])

## Train
### Linear Regression
##### Fixed Step Size = 0.01

In [ ]:
from sklearn.linear_model import SGDRegressor

fixedRegressor = SGDRegressor(loss='squared_loss', penalty='None', fit_intercept=True, shuffle=True,
                         learning_rate='constant', eta0=0.01, max_iter=100)
fixedRegressor.fit(Xtrain, Ytrain)

##### Inversely Scaled Step Size

In [ ]:
invRegressor = SGDRegressor(loss='squared_loss', penalty='None', fit_intercept=True, shuffle=True,
                            learning_rate='invscaling', max_iter=100)
invRegressor.fit(Xtrain, Ytrain)

### Ridge Regression
##### Fixed Step Size = 0.01

In [ ]:
fixedL2Regressor = SGDRegressor(loss='squared_loss', penalty='l2', fit_intercept=True, shuffle=True,
                            learning_rate='constant', max_iter=100, eta0=0.01)
fixedL2Regressor.fit(Xtrain, Ytrain)

##### Inversely Scaled Step Size

In [ ]:
invL2Regressor = SGDRegressor(loss='squared_loss', penalty='l2', fit_intercept=True, shuffle=True,
                            learning_rate='invscaling', max_iter=100)
invL2Regressor.fit(Xtrain, Ytrain)

### Lasso Regression
##### Fixed Step Size = 0.01

In [ ]:
fixedL1Regressor = SGDRegressor(loss='squared_loss', penalty='l1', fit_intercept=True, shuffle=True,
                            learning_rate='constant', max_iter=100, eta0=0.01)
fixedL1Regressor.fit(Xtrain, Ytrain)

##### Inversely Scaled Step Size

In [ ]:
invL1Regressor = SGDRegressor(loss='squared_loss', penalty='l1', fit_intercept=True, shuffle=True,
                            learning_rate='invscaling', max_iter=100)
invL1Regressor.fit(Xtrain, Ytrain)


## Polynomial Regression
##### Transform data

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(2)
Xtrain = poly.fit_transform(Xtrain)
Xtest = poly.fit_transform(Xtest)

types = []

##### Fixed Step Size = 0.01

In [ ]:
from sklearn.linear_model import SGDRegressor

polyFixedReg = SGDRegressor(loss='squared_loss', penalty='None', fit_intercept=True, shuffle=True,
                         learning_rate='constant', eta0=0.01, max_iter=100)
polyFixedReg.fit(Xtrain, Ytrain)

types.append(('Polynomial Regression with fixed step size', polyFixedReg))

##### Inversely Scaled Step Size

In [ ]:
polyInvReg = SGDRegressor(loss='squared_loss', penalty='None', fit_intercept=True, shuffle=True,
                            learning_rate='invscaling', max_iter=100)
polyInvReg.fit(Xtrain, Ytrain)

types.append(('Polynomial Regression with inversely scaled step size', polyInvReg))

### Polynomial Ridge Regression
##### Fixed Step Size = 0.01

In [ ]:
polyFixedL2Reg = SGDRegressor(loss='squared_loss', penalty='l2', fit_intercept=True, shuffle=True,
                            learning_rate='constant', max_iter=100, eta0=0.01)
polyFixedL2Reg.fit(Xtrain, Ytrain)

types.append(('Polynomial Ridge Regression with fixed step size', polyFixedL2Reg))

##### Inversely Scaled Step Size

In [ ]:
polyInvL2Reg = SGDRegressor(loss='squared_loss', penalty='l2', fit_intercept=True, shuffle=True,
                            learning_rate='invscaling', max_iter=100)
polyInvL2Reg.fit(Xtrain, Ytrain)

types.append(('Polynomial Ridge Regression with inversely scaled step size', polyInvL2Reg))

## Polynomial Lasso Regression
##### Fixed Step Size = 0.01

In [ ]:
polyFixedL1Reg = SGDRegressor(loss='squared_loss', penalty='l1', fit_intercept=True, shuffle=True,
                            learning_rate='constant', max_iter=100, eta0=0.01)
polyFixedL1Reg.fit(Xtrain, Ytrain)

types.append(('Polynomial Lasso Regression with a fixed step size', polyFixedL1Reg))

##### Inversely Scaled Step Size

In [ ]:
polyInvL1Reg = SGDRegressor(loss='squared_loss', penalty='l1', fit_intercept=True, shuffle=True,
                            learning_rate='invscaling', max_iter=100)
polyInvL1Reg.fit(Xtrain, Ytrain)

types.append(('Polynomial Lasso Regression with inversely scaled step size', polyInvL1Reg))

## Predict

In [ ]:
from sklearn.dummy import DummyRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error

dummy = DummyRegressor(strategy='mean')
dummy.fit(Xtrain, Ytrain)
dummyPredictions = dummy.predict(Xtest)
print('Mean Error:', np.square(np.subtract(dummyPredictions, Ytest)).mean())

for regressor in types:
    predictions = regressor[1].predict(Xtest)
    scores = cross_val_score(regressor[1], Xtrain, Ytrain, cv=5)
    print(f"{scores.mean()} accuracy with standard deviation of {scores.std()}")
    print('Mean Squared Error for ' + regressor[0] + ':', mean_squared_error(predictions, Ytest))
    print('')




